<a href="https://colab.research.google.com/github/HazemmoAlsady/AWN_Graduation_Project/blob/main/chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install \
transformers==4.38.2 \
sentence-transformers==2.6.1 \
langchain==0.1.16 \
langchain-community==0.0.34 \
langchain-core==0.1.45 \
langchain-text-splitters \
faiss-cpu \
accelerate


In [2]:
import os

os.makedirs("docs", exist_ok=True)


In [3]:
with open("docs/register.md", "w", encoding="utf-8") as f:
    f.write("""
# التسجيل في منصة عون

## خطوات التسجيل
- الدخول على موقع عون
- إنشاء حساب جديد
- إدخال البيانات الأساسية
- تأكيد رقم الهاتف

## ملاحظات
- يجب إدخال بيانات صحيحة
- رقم الهاتف إلزامي
""")


In [4]:
with open("docs/case_submit.md", "w", encoding="utf-8") as f:
    f.write("""
# رفع حالة في منصة عون

## خطوات رفع الحالة
- تسجيل الدخول إلى الحساب
- اختيار رفع حالة جديدة
- إدخال بيانات الأسرة
- إرفاق المستندات المطلوبة
- إرسال الطلب للمراجعة

## ملاحظات
- يجب إرفاق مستندات واضحة
- الطلب يخضع للمراجعة
""")


In [5]:
with open("docs/terms.md", "w", encoding="utf-8") as f:
    f.write("""
# شروط الاستفادة من منصة عون

- أن تكون البيانات صحيحة
- أن تكون الحالة حقيقية
- تقديم المستندات المطلوبة
- قبول سياسة الاستخدام
""")


In [6]:
with open("docs/faq.md", "w", encoding="utf-8") as f:
    f.write("""
# الأسئلة الشائعة

## هل التسجيل مجاني؟
نعم، التسجيل مجاني تمامًا.

## هل يمكن تعديل البيانات؟
نعم، يمكن تعديل البيانات قبل إرسال الطلب.
""")


In [7]:
!ls docs


case_submit.md	faq.md	register.md  terms.md


In [8]:
from langchain_community.document_loaders import TextLoader

files = [
    "docs/register.md",
    "docs/case_submit.md",
    "docs/terms.md",
    "docs/faq.md"
]

documents = []
for file in files:
    loader = TextLoader(file, encoding="utf-8")
    documents.extend(loader.load())

print("عدد الملفات:", len(documents))


عدد الملفات: 4


In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50
)

chunks = splitter.split_documents(documents)
print("عدد القطع:", len(chunks))


عدد القطع: 4


In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

vector_db = FAISS.from_documents(chunks, embeddings)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possibl

In [11]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

pipe = pipeline(
    task="text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=200
)

llm = HuggingFacePipeline(pipeline=pipe)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [28]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt = PromptTemplate.from_template("""
أنت مساعد رسمي لمنصة عون.
أجب بلغة عربية بسيطة وواضحة.
اعتمد فقط على المعلومات الموجودة في السياق.
إذا لم تجد إجابة، قل:
"المعلومة دي غير متاحة حاليًا في منصة عون."

السياق:
{context}

السؤال:
{question}

الإجابة:
""")

retriever = vector_db.as_retriever(search_kwargs={"k": 3})

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [29]:
rag_chain.invoke("إزاي أسجل في منصة عون؟")


'                                                                                                   '

In [20]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.9 MB/s eta 0:00:00


In [21]:
import streamlit as st

st.set_page_config(page_title="مساعد منصة عون", layout="centered")
st.title("🤖 مساعد منصة عون")

question = st.text_input("اسأل عن منصة عون")

if question:
    answer = rag_chain.invoke(question)
    st.write(answer)


2026-02-01 12:48:55.971 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-01 12:48:55.972 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-01 12:48:56.513 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2026-02-01 12:48:56.515 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-01 12:48:56.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-01 12:48:56.522 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-01 12:48:56.526 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn